In [ ]:
# call another jupyter notebook to run required function
%run 'MyCanny_script.ipynb'

import kornia
import torch

def MyCannyFull(input_img:torch.Tensor, st_deviation:float, gm_threshold:float, low_gm_threshold:float, high_gm_threshold:float) -> torch.Tensor:
    # apply Canny edge detection from MyCanny function
    nms_img, my_canny_img = MyCanny(input_img, st_deviation, gm_threshold)

    # store discovered edges
    output_img = torch.zeros_like(nms_img)
    
    # queue for BFS
    queue = []

    # mark pixels above the high threshold as edges and add them to the queue
    high_threshold_mask = nms_img > high_gm_threshold
    output_img[high_threshold_mask] = 1
    queue.extend(torch.nonzero(high_threshold_mask, as_tuple=False).tolist())

    # perform BFS to connect edges
    while queue:
        select_p = queue.pop(0)
        row, col = select_p[0], select_p[1]
        
        # run through neighbour pixels
        for i in range(-1, 2):
            for j in range(-1, 2):
                if 0 <= row + i < nms_img.shape[0] and 0 <= col + j < nms_img.shape[1]:
                    if (output_img[row + i, col + j] == 0 and nms_img[row + i, col + j] > low_gm_threshold):
                        output_img[row + i, col + j] = 1
                        queue.append((row + i, col + j))

    return output_img